# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        img -= np.mean(img)
        img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return images
    #return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [7]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print (Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [8]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 8.2131 Disc Loss: 1.37209 Q Losses: [0.21092363, 2.3025088]
epoch:0 batch_done:2 Gen Loss: 6.17389 Disc Loss: 0.304182 Q Losses: [0.15146941, 2.317749]
epoch:0 batch_done:3 Gen Loss: 14.2564 Disc Loss: 0.789707 Q Losses: [0.14752175, 2.2731493]
epoch:0 batch_done:4 Gen Loss: 7.94339 Disc Loss: 1.19521 Q Losses: [0.1438432, 2.2646496]
epoch:0 batch_done:5 Gen Loss: 9.07372 Disc Loss: 0.443367 Q Losses: [0.12777501, 2.2597294]
epoch:0 batch_done:6 Gen Loss: 12.1364 Disc Loss: 0.364585 Q Losses: [0.12539355, 2.2745388]
epoch:0 batch_done:7 Gen Loss: 8.86962 Disc Loss: 0.354851 Q Losses: [0.10812667, 2.2230153]
epoch:0 batch_done:8 Gen Loss: 13.2356 Disc Loss: 0.409395 Q Losses: [0.096717626, 2.2638135]
epoch:0 batch_done:9 Gen Loss: 8.82091 Disc Loss: 0.319841 Q Losses: [0.10619939, 2.2623992]
epoch:0 batch_done:10 Gen Loss: 22.57 Disc Loss: 2.33908 Q Losses: [0.13083716, 2.2167377]
epoch:0 batch_done:11 Gen Loss: 14.3662 Disc Loss: 3.02198 Q Losses: [0.1185

epoch:0 batch_done:89 Gen Loss: 37.6401 Disc Loss: 2.23217 Q Losses: [0.04707266, 0.98472559]
epoch:0 batch_done:90 Gen Loss: 42.3305 Disc Loss: 1.74594 Q Losses: [0.039180789, 1.0295266]
epoch:0 batch_done:91 Gen Loss: 35.7975 Disc Loss: 0.846649 Q Losses: [0.0448916, 0.99979115]
epoch:0 batch_done:92 Gen Loss: 23.7254 Disc Loss: 0.241562 Q Losses: [0.065734081, 0.99838662]
epoch:0 batch_done:93 Gen Loss: 8.10042 Disc Loss: 0.0290089 Q Losses: [0.061919324, 0.9476403]
epoch:0 batch_done:94 Gen Loss: 38.8562 Disc Loss: 3.10884 Q Losses: [0.063290648, 0.95101571]
epoch:0 batch_done:95 Gen Loss: 42.266 Disc Loss: 2.27878 Q Losses: [0.053678751, 0.92767]
epoch:0 batch_done:96 Gen Loss: 32.8427 Disc Loss: 2.11764 Q Losses: [0.064074911, 0.9326942]
epoch:0 batch_done:97 Gen Loss: 21.1186 Disc Loss: 1.12305 Q Losses: [0.044609159, 0.87461638]
epoch:0 batch_done:98 Gen Loss: 8.64448 Disc Loss: 0.282854 Q Losses: [0.055580292, 0.91853905]
epoch:0 batch_done:99 Gen Loss: 6.57475 Disc Loss: 0.11

epoch:0 batch_done:175 Gen Loss: 22.5981 Disc Loss: 0.263855 Q Losses: [0.036903683, 0.38455388]
epoch:0 batch_done:176 Gen Loss: 17.2474 Disc Loss: 0.0936861 Q Losses: [0.048709266, 0.38559812]
epoch:0 batch_done:177 Gen Loss: 10.3501 Disc Loss: 0.0857817 Q Losses: [0.030798223, 0.38668928]
epoch:0 batch_done:178 Gen Loss: 4.59664 Disc Loss: 0.0113896 Q Losses: [0.037000649, 0.38251594]
epoch:0 batch_done:179 Gen Loss: 25.4686 Disc Loss: 0.819543 Q Losses: [0.036630422, 0.37490487]
epoch:0 batch_done:180 Gen Loss: 27.9771 Disc Loss: 0.718237 Q Losses: [0.030131966, 0.38719541]
epoch:0 batch_done:181 Gen Loss: 22.0718 Disc Loss: 0.758681 Q Losses: [0.032661304, 0.35737544]
epoch:0 batch_done:182 Gen Loss: 11.4459 Disc Loss: 0.26377 Q Losses: [0.028575521, 0.39053723]
epoch:0 batch_done:183 Gen Loss: 7.80466 Disc Loss: 0.431643 Q Losses: [0.02241639, 0.35371113]
epoch:0 batch_done:184 Gen Loss: 33.4672 Disc Loss: 0.839464 Q Losses: [0.037517965, 0.37651777]
epoch:0 batch_done:185 Gen Lo

epoch:1 batch_done:54 Gen Loss: 4.83376 Disc Loss: 0.0192062 Q Losses: [0.02659083, 0.22239712]
epoch:1 batch_done:55 Gen Loss: 17.8201 Disc Loss: 0.279992 Q Losses: [0.023991738, 0.20646657]
epoch:1 batch_done:56 Gen Loss: 17.1679 Disc Loss: 0.344067 Q Losses: [0.01993876, 0.20467721]
epoch:1 batch_done:57 Gen Loss: 11.6715 Disc Loss: 0.250682 Q Losses: [0.023655117, 0.21466273]
epoch:1 batch_done:58 Gen Loss: 6.39291 Disc Loss: 0.0188015 Q Losses: [0.020906715, 0.20451525]
epoch:1 batch_done:59 Gen Loss: 5.43899 Disc Loss: 0.0653448 Q Losses: [0.022125553, 0.23193613]
epoch:1 batch_done:60 Gen Loss: 8.14117 Disc Loss: 0.111848 Q Losses: [0.021342019, 0.20347723]
epoch:1 batch_done:61 Gen Loss: 7.46284 Disc Loss: 0.0333681 Q Losses: [0.018586241, 0.18574303]
epoch:1 batch_done:62 Gen Loss: 5.65004 Disc Loss: 0.0739051 Q Losses: [0.025950348, 0.1952962]
epoch:1 batch_done:63 Gen Loss: 5.32116 Disc Loss: 0.258503 Q Losses: [0.029974639, 0.18559192]
epoch:1 batch_done:64 Gen Loss: 7.3384

epoch:1 batch_done:139 Gen Loss: 6.83508 Disc Loss: 0.00694096 Q Losses: [0.012174679, 0.13742769]
epoch:1 batch_done:140 Gen Loss: 26.6268 Disc Loss: 0.462159 Q Losses: [0.017606799, 0.13781062]
epoch:1 batch_done:141 Gen Loss: 25.1827 Disc Loss: 0.572105 Q Losses: [0.017610425, 0.15117589]
epoch:1 batch_done:142 Gen Loss: 18.18 Disc Loss: 0.342289 Q Losses: [0.01922103, 0.12414168]
epoch:1 batch_done:143 Gen Loss: 12.4271 Disc Loss: 0.00969673 Q Losses: [0.017137645, 0.13171029]
epoch:1 batch_done:144 Gen Loss: 9.0906 Disc Loss: 0.0033722 Q Losses: [0.019204564, 0.14269292]
epoch:1 batch_done:145 Gen Loss: 5.78753 Disc Loss: 0.0211001 Q Losses: [0.020754416, 0.13503754]
epoch:1 batch_done:146 Gen Loss: 4.42157 Disc Loss: 0.061822 Q Losses: [0.017410165, 0.12977621]
epoch:1 batch_done:147 Gen Loss: 5.23502 Disc Loss: 0.0273883 Q Losses: [0.015736038, 0.12045904]
epoch:1 batch_done:148 Gen Loss: 6.08079 Disc Loss: 0.0129936 Q Losses: [0.013553713, 0.12020253]
epoch:1 batch_done:149 Gen

...

epoch:9 batch_done:145 Gen Loss: 37.8244 Disc Loss: 6.66723 Q Losses: [0.011655685, 0.056860581]
epoch:9 batch_done:146 Gen Loss: 25.4813 Disc Loss: 3.53903e-08 Q Losses: [0.025982991, 0.086227208]
epoch:9 batch_done:147 Gen Loss: 10.5899 Disc Loss: 2.84053e-07 Q Losses: [0.017934937, 0.032803558]
epoch:9 batch_done:148 Gen Loss: 5.66344 Disc Loss: 0.0455412 Q Losses: [0.012451663, 0.072566435]
epoch:9 batch_done:149 Gen Loss: 34.6416 Disc Loss: 0.421503 Q Losses: [0.016589776, 0.030576553]
epoch:9 batch_done:150 Gen Loss: 42.151 Disc Loss: 7.45059e-08 Q Losses: [0.014332025, 0.036939621]
epoch:9 batch_done:151 Gen Loss: 42.385 Disc Loss: 9.56894e-06 Q Losses: [0.012848741, 0.025357768]
epoch:9 batch_done:152 Gen Loss: 41.3832 Disc Loss: 0.000119033 Q Losses: [0.010709639, 0.018213496]
epoch:9 batch_done:153 Gen Loss: 35.3763 Disc Loss: 0.0711736 Q Losses: [0.01426388, 0.018809423]
epoch:9 batch_done:154 Gen Loss: 27.4208 Disc Loss: 0.0317273 Q Losses: [0.015468718, 0.018615797]
epoch:

epoch:10 batch_done:20 Gen Loss: 38.1876 Disc Loss: 0.00030841 Q Losses: [0.011689628, 0.0083836112]
epoch:10 batch_done:21 Gen Loss: 34.968 Disc Loss: 0.0506349 Q Losses: [0.0095015205, 0.0050320458]
epoch:10 batch_done:22 Gen Loss: 25.6092 Disc Loss: 0.263864 Q Losses: [0.0090417704, 0.0054139579]
epoch:10 batch_done:23 Gen Loss: 15.4216 Disc Loss: 0.160865 Q Losses: [0.0080779269, 0.0059104371]
epoch:10 batch_done:24 Gen Loss: 9.17992 Disc Loss: 0.0494824 Q Losses: [0.010069171, 0.0093113296]
epoch:10 batch_done:25 Gen Loss: 6.11138 Disc Loss: 0.0623561 Q Losses: [0.012445221, 0.0087383017]
epoch:10 batch_done:26 Gen Loss: 9.41524 Disc Loss: 0.0109306 Q Losses: [0.0079310052, 0.0069516157]
epoch:10 batch_done:27 Gen Loss: 6.9598 Disc Loss: 0.0417436 Q Losses: [0.012005714, 0.0078403987]
epoch:10 batch_done:28 Gen Loss: 37.3004 Disc Loss: 0.658504 Q Losses: [0.0092174998, 0.0086986618]
epoch:10 batch_done:29 Gen Loss: 36.7272 Disc Loss: 1.69268 Q Losses: [0.012267983, 0.0078992266]
e

epoch:10 batch_done:103 Gen Loss: 6.23786 Disc Loss: 0.0883282 Q Losses: [0.0059392368, 0.013398651]
epoch:10 batch_done:104 Gen Loss: 6.5291 Disc Loss: 0.152524 Q Losses: [0.030552872, 0.0072561372]
epoch:10 batch_done:105 Gen Loss: 3.54774 Disc Loss: 0.345355 Q Losses: [0.016483136, 0.015263208]
epoch:10 batch_done:106 Gen Loss: 36.5614 Disc Loss: 0.641014 Q Losses: [0.010679618, 0.0083126677]
epoch:10 batch_done:107 Gen Loss: 34.5585 Disc Loss: 2.09326 Q Losses: [0.010363664, 0.0093186991]
epoch:10 batch_done:108 Gen Loss: 27.5967 Disc Loss: 0.499489 Q Losses: [0.00795775, 0.0063536097]
epoch:10 batch_done:109 Gen Loss: 20.351 Disc Loss: 0.172108 Q Losses: [0.013495372, 0.0070690531]
epoch:10 batch_done:110 Gen Loss: 15.2071 Disc Loss: 4.83429e-05 Q Losses: [0.0045672911, 0.0093449885]
epoch:10 batch_done:111 Gen Loss: 10.6539 Disc Loss: 0.000203818 Q Losses: [0.0079351887, 0.01019518]
epoch:10 batch_done:112 Gen Loss: 6.01614 Disc Loss: 0.0674292 Q Losses: [0.0080192611, 0.01314854

epoch:10 batch_done:189 Gen Loss: nan Disc Loss: nan Q Losses: [0.1927771, 2.3015885]
epoch:10 batch_done:190 Gen Loss: nan Disc Loss: nan Q Losses: [0.18376932, 2.304894]
epoch:10 batch_done:191 Gen Loss: nan Disc Loss: nan Q Losses: [0.18328318, 2.3026342]
epoch:10 batch_done:192 Gen Loss: nan Disc Loss: nan Q Losses: [0.14758044, 2.301434]
epoch:10 batch_done:193 Gen Loss: nan Disc Loss: nan Q Losses: [0.2000674, 2.3025129]
epoch:10 batch_done:194 Gen Loss: nan Disc Loss: nan Q Losses: [0.16240129, 2.3042691]
epoch:10 batch_done:195 Gen Loss: nan Disc Loss: nan Q Losses: [0.15483439, 2.3043442]
epoch:10 batch_done:196 Gen Loss: nan Disc Loss: nan Q Losses: [0.18043047, 2.3029013]
epoch:10 batch_done:197 Gen Loss: nan Disc Loss: nan Q Losses: [0.15961009, 2.3022304]
epoch:10 batch_done:198 Gen Loss: nan Disc Loss: nan Q Losses: [0.15536895, 2.3061495]
epoch:10 batch_done:199 Gen Loss: nan Disc Loss: nan Q Losses: [0.15462056, 2.3016303]
epoch:10 batch_done:200 Gen Loss: nan Disc Loss

KeyboardInterrupt: 

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 